In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')
df.head()

In [ ]:
df = df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
              'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], 
             axis=1)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.drop('CLIENTNUM', axis=1)
df.head()

In [ ]:
df['Attrition_Flag'].unique()

In [ ]:
df['churn']= df['Attrition_Flag'].map({'Existing Customer':1,'Attrited Customer':0})
df = df.drop('Attrition_Flag', axis=1)
df.head()

In [ ]:
df['churn'].value_counts().plot(kind='bar')

it is imbalanced

In [ ]:
df['Income_Category'].unique()

In [ ]:
df['Income_Category'].value_counts()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df['Gender'].unique()

In [ ]:
fig, ax = plt.subplots()

sns.catplot("Gender", hue="churn", data=df, kind="count", 
            palette={1:"blue", 0:"green"}, ax=ax)
plt.close(1)

gender is not a good way to know churners

In [ ]:
fig, ax = plt.subplots()

sns.catplot("Income_Category", hue="churn", data=df, kind="count", 
            palette={1:"blue", 0:"green"}, ax=ax)
plt.xticks(rotation=70)
plt.close(1)

In [ ]:
df['Income_Category']= df['Income_Category'].replace(['Unknown'], 
                                                     'NaN')

In [ ]:
fig, ax = plt.subplots()

sns.catplot("Income_Category", hue="churn", data=df, kind="count", 
            palette={1:"blue", 0:"green"}, ax=ax)
plt.xticks(rotation=70)
plt.close(1)

In [ ]:
df = df.drop(df[df['Income_Category'] == 'NaN'].index)

In [ ]:
fig, ax = plt.subplots()

sns.catplot("Income_Category", hue="churn", data=df, kind="count", 
            palette={1:"blue", 0:"green"}, ax=ax)
plt.xticks(rotation=70)
plt.close(1)

INCOME CATEGORY IS NOT A GOOD WAY TO PREDICT THE CHURN

In [ ]:
fig, ax = plt.subplots()

sns.catplot("Education_Level", hue="churn", data=df, kind="count", 
            palette={1:"blue", 0:"green"}, ax=ax)
plt.xticks(rotation=70)
plt.close(1)

Education level is not also a good predictor

In [ ]:
df = df.drop(df[df['Income_Category'] == 'Unknown'].index)

In [ ]:
fig, ax = plt.subplots()

sns.catplot("Marital_Status", hue="churn", data=df, kind="count", 
            palette={1:"blue", 0:"green"}, ax=ax)
plt.xticks(rotation=70)
plt.close(1)

In [ ]:
df = df.drop(df[df['Marital_Status'] == 'Unknown'].index)

marital status is not a good predictor of churn

In [ ]:
df.corr()['churn'][:-1].sort_values().plot(kind='bar')

In [ ]:
df['gender']= df['Gender'].map({'M':1,'F':0})
df = df.drop('Gender', axis=1)
df.head()

In [ ]:
df.select_dtypes(['object']).columns

In [ ]:
fig, ax = plt.subplots()

sns.catplot("Card_Category", hue="churn", data=df, kind="count", 
            palette={1:"blue", 0:"green"}, ax=ax)
plt.xticks(rotation=70)
plt.close(1)

even though there are few people on thee Gold category very few are likely to churn 
followed by the silver then Blue

In [ ]:
dummies = pd.get_dummies(df[['Education_Level', 'Marital_Status', 'Income_Category','Card_Category']], drop_first=True)
df = pd.concat([df.drop(['Education_Level', 'Marital_Status', 'Income_Category','Card_Category'], axis=1), dummies], axis=1)

In [ ]:
df.head()

In [ ]:
import xgboost as xgb

In [ ]:
df1 = df
df1.head()

In [ ]:
X = df1.drop('churn', axis=1).values
y = df1['churn'].values

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=101)

In [ ]:
model = xgb.XGBClassifier(objective='binary:logistic')
param_grid = {'colsample_bytree': [0.3, 0.7],
                  'n_estimators': [50],
                  'max_depth': [2, 5]}

In [ ]:
model_roc_auc= RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                                        n_iter=10,
                                        scoring='roc_auc', cv=4)

In [ ]:
model_roc_auc.fit(X_train, y_train)

In [ ]:
y_pred = model_roc_auc.predict(X_test)

In [ ]:
accuracy = float(np.sum(y_pred==y_test))/y_test.shape[0]

In [ ]:
print('accuracy: %f' %(accuracy))

In [ ]:
churn_new = df1.drop('churn', axis=1).iloc[0]

In [ ]:
churn_new

In [ ]:
churn_new1 = churn_new.values.reshape(-1, 30)

In [ ]:
model_roc_auc.predict(churn_new1)

In [ ]:
df1.head(1)

referenced from Huynh Dong Nguyen

In [ ]:
def Definedata():
    # define dataset
    X = df1.drop('churn', axis=1).values
    y = df1['churn'].values
    return X, y

In [ ]:
def SMOTE():
    # borderline-SMOTE for imbalanced dataset
    from collections import Counter
    from sklearn.model_selection import train_test_split
    from sklearn.datasets import make_classification
    from imblearn.over_sampling import SMOTE
    from matplotlib import pyplot
    from numpy import where
    
    X, y = Definedata()

# summarize class distribution
    counter = Counter(y)
    print(counter)
# transform the dataset
    smt = SMOTE(random_state=0)
    X, y = smt.fit_sample(X, y) 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)
# summarize the new class distribution
    counter = Counter(y)
    print(counter)
# scatter plot of examples by class label
    for label, _ in counter.items():
        row_ix = where(y == label)[0]
        pyplot.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
    pyplot.legend()
    pyplot.show()
    return X_train, X_test, y_train, y_test

In [ ]:
def ADASYN():
    from collections import Counter
    from sklearn.model_selection import train_test_split
    from imblearn.over_sampling import ADASYN
    from matplotlib import pyplot
    from numpy import where

    X, y = Definedata()

# summarize class distribution
    counter = Counter(y)
    print(counter)
# transform the dataset
    X, y = ADASYN().fit_resample(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)
# summarize the new class distribution
    counter = Counter(y)
    print(counter)
# scatter plot of examples by class label
    for label, _ in counter.items():
        row_ix = where(y == label)[0]
        pyplot.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
    pyplot.legend()
    pyplot.show()
    return X_train, X_test, y_train, y_test


In [ ]:
%time X_train1, X_test1, y_train1, y_test1 = SMOTE()
%time X_train4, X_test4, y_train4, y_test4 = ADASYN()

In [ ]:
def Models(models, X_train, X_test, y_train, y_test, title, graph):
    model = models
    model.fit(X_train,y_train)
    
    X, y = Definedata()
    train_matrix = pd.crosstab(y_train, model.predict(X_train), rownames=['Actual'], colnames=['Predicted'])    
    test_matrix = pd.crosstab(y_test, model.predict(X_test), rownames=['Actual'], colnames=['Predicted'])
    matrix = pd.crosstab(y, model.predict(X), rownames=['Actual'], colnames=['Predicted'])
    
    if graph:
        f,(ax1,ax2,ax3) = plt.subplots(1,3,sharey=True, figsize=(15, 2))
    
        g1 = sns.heatmap(train_matrix, annot=True, fmt=".1f", cbar=False,annot_kws={"size": 18},ax=ax1)
        g1.set_title(title)
        g1.set_ylabel('Total Churn = {}'.format(y_train.sum()), fontsize=14, rotation=90)
        g1.set_xlabel('Accuracy score (TrainSet): {}'.format(accuracy_score(model.predict(X_train), y_train)))
        g1.set_xticklabels(['Churn','Not Churn'],fontsize=12)

        g2 = sns.heatmap(test_matrix, annot=True, fmt=".1f",cbar=False,annot_kws={"size": 18},ax=ax2)
        g2.set_title(title)
        g2.set_ylabel('Total Churn = {}'.format(y_test.sum()), fontsize=14, rotation=90)
        g2.set_xlabel('Accuracy score (TestSet): {}'.format(accuracy_score(model.predict(X_test), y_test)))
        g2.set_xticklabels(['Churn','Not Churn'],fontsize=12)
        g3 = sns.heatmap(matrix, annot=True, fmt=".1f",cbar=False,annot_kws={"size": 18},ax=ax3)
        g3.set_title(title)
        g3.set_ylabel('Total Churn = {}'.format(y.sum()), fontsize=14, rotation=90)
        g3.set_xlabel('Accuracy score (Total): {}'.format(accuracy_score(model.predict(X), y)))
        g3.set_xticklabels(['Churn','Not Churn'],fontsize=12)

        plt.show()

    print("\t\tError Table")
    print('Accuracy on Traing set   : ', model.score(X_train,y_train))
    print('Accuracy on Testing set  : ', model.score(X_test,y_test))
    print('Overall Accuracy_Score   :',accuracy_score(y, model.predict(X))*100,'%')
    print('Recall ratio             :',metrics.recall_score(y, model.predict(X))*100,'%')
    print('AUC score                :', roc_auc_score(y, model.predict(X))*100,'%')

    return y, model.predict(X)

In [ ]:
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, roc_curve

In [ ]:
title = 'XGBClassifier/SMOTE'
%time Models(XGBClassifier(colsample_bytree=0.9, learning_rate=0.2, max_depth=7),X_train1, X_test1, y_train1, y_test1, title, True)

In [ ]:
title = 'XGBClassifier/ADASYN'
%time Models(XGBClassifier(colsample_bytree=0.9, learning_rate=0.2, max_depth=7),X_train4, X_test4, y_train4, y_test4, title, True)